## 차원축소
- 매우 많은 피처로 구성된 다차원 데이터 세트의 차원을 축소해 새로운 차원의 데이터 세트를 생성  
- 필요성   
    - 차원이 증가할수록 데이터 포인트 간의 거리가 멀어져 희소한 구조를 가지게되어 예측 신뢰도가 떨어집니다.
    - 피처가 많을 경우 개별 피처간에 상관관계가 높은 가능성이 크며, 선형 회귀 같은 선형 모델에서는 입력 변수 간의 상관관계가 높을 경우 이로 인한 다중 공선성으로 예측 성능이 저하됩니다.
    - 이렇게 매우 많은 다차원의 피처를 차원 축소해 피처 수를 줄이면 더 직관적으로 데이터를 해석할 수 있기 때문에 차원축소는 중요합니다.